In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from fbprophet import Prophet

from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

import datetime

from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor 
import shap
from catboost import Pool



In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#train
data = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/train.csv", low_memory=False)#, nrows=10000)

#test
test = pd.read_csv("/kaggle/input/tabular-playground-series-jul-2021/test.csv", low_memory=False)

In [ ]:
test

In [ ]:
data.describe().transpose()

In [ ]:
data.info()

# **Heat Map**

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,8))
sns.heatmap(data.corr(method = 'pearson'), annot=True, ax=ax, cmap='Blues_r')
plt.show()

> feature sensor_1, sensor_2, sensor_4, sensor_5 have good +ve corelation with all 3 feature
> feature sensor_3 has good -ve corelation too.
> This could turnout to very useful feature in model.

> Let try see on scattern plot

In [ ]:
sns.pairplot(data[['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']])

# EDA

In [ ]:
# Typecast datetime form object to datetime field and make index
data.index = pd.to_datetime(data.pop('date_time'))
test.index = pd.to_datetime(test.pop('date_time'))

In [ ]:
#Ploting target field on timeline
targets = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']
for t in targets:
    my_alpha=0.25
    fig, ax = plt.subplots(figsize=(12,3))
    ax.scatter(data.index, data[t], alpha=my_alpha, 
               color='darkred', s=4)
    ax.xaxis.set_major_locator(plt.MaxNLocator(20)) # reduce number of x-axis labels
    plt.xticks(rotation=90)
    plt.title(t)
    plt.grid()
    plt.tight_layout()
plt.show()

We can divide target in 3 part basis of pattern.
1. Before Aug'10
2. Aug'10
3. After Aug'10

Let see timeline plot of features with test data.
Blue is showing train data and green is test date

In [ ]:
features_num = test.columns
for f in features_num:
    plt.figure(figsize=(14,4))
    ax1 = plt.subplot(2,1,1)
    ax1.scatter(data.index, data[f], alpha=my_alpha, color='blue', s=4)
    ax1.scatter(test.index, test[f], alpha=my_alpha, color='green', s=4)
    plt.title(f)
    plt.grid()

Now let's check Autocorrelation

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(data['target_carbon_monoxide'], lags=200)

**The scalloped shape is season and decrease pattern is incidation of trend.**
We can see treand at 24(at day) and 174(at week)

Let also see for other 2 targets

In [ ]:
plot_acf(data['target_benzene'], lags=200)

In [ ]:
plot_acf(data['target_nitrogen_oxides'], lags=200)

The Pattern do exist. Yeah!!

I have use Prophet to get trend and seasonlity as feature

Prophet for target_benzene

In [ ]:
# Prophet for target_benzene
df = data[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_benzene']]
df = df.reset_index(level=0)
df.columns = ['ds','deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'y']
p = Prophet(seasonality_mode='additive')
p.fit(df)
future = p.make_future_dataframe(periods=2246, freq='h')
future['floor'] =  0 
forecast = p.predict(future)

# Add prediction on train data
data = data.merge(forecast[['ds','yhat']][:-2246], how="inner", right_on="ds", left_index=True)
data = data.rename(columns={'yhat':'yhat_benzene_oxide'})
data.index = data.pop('ds')

# Add prediction on test data
test = test.merge(forecast[['ds','yhat']][-2246:], how="inner", right_on="ds", left_index=True)
test = test.rename(columns={'yhat':'yhat_benzene_oxide'})
test.index = test.pop('ds')

Prophet for target_carbon_monoxide

In [ ]:
# Prophet for target_carbon_monoxide
df = data[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_carbon_monoxide']]
df = df.reset_index(level=0)
df.columns = ['ds','deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'y']
p = Prophet(seasonality_mode='additive')
p.fit(df)
future = p.make_future_dataframe(periods=2246, freq='h')
future['floor'] =  0 
forecast = p.predict(future)

# Add prediction on train data
data = data.merge(forecast[['ds','yhat']][:-2246], how="inner", right_on="ds", left_index=True)
data = data.rename(columns={'yhat':'yhat_monoxide'})
data.index = data.pop('ds')

# Add prediction on test data
test = test.merge(forecast[['ds','yhat']][-2246:], how="inner", right_on="ds", left_index=True)
test = test.rename(columns={'yhat':'yhat_monoxide'})
test.index = test.pop('ds')

Prophet for target_nitrogen_oxides

In [ ]:
# Prophet for target_nitrogen_oxides
df = data[['deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'target_nitrogen_oxides']]
df = df.reset_index(level=0)
df.columns = ['ds','deg_C', 'relative_humidity', 'absolute_humidity', 'sensor_1',
       'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       'y']
p = Prophet(seasonality_mode='additive')
p.fit(df)
future = p.make_future_dataframe(periods=2246, freq='h')
future['floor'] =  0 
forecast = p.predict(future)

# Add prediction on train data
data = data.merge(forecast[['ds','yhat']][:-2246], how="inner", right_on="ds", left_index=True)
data = data.rename(columns={'yhat':'yhat_nitrogen_oxide'})
data.index = data.pop('ds')

# Add prediction on test data
test = test.merge(forecast[['ds','yhat']][-2246:], how="inner", right_on="ds", left_index=True)
test = test.rename(columns={'yhat':'yhat_nitrogen_oxide'})
test.index = test.pop('ds')

In [ ]:
# Make cyclic feature for time
timestamp_s = data.index.map(datetime.datetime.timestamp)
timestamp_t_s = test.index.map(datetime.datetime.timestamp)

day = 24*60*60
week = day * 7

# 1 day
data['Day sin'] = np.sin(timestamp_s * (2*np.pi/day))
data['Day cos'] = np.cos(timestamp_s * (2*np.pi/day))

# 1 week
data['Week sin'] = np.sin(timestamp_s * (2*np.pi/week))
data['Week cos'] = np.cos(timestamp_s * (2*np.pi/week))


# 1 day
test['Day sin'] = np.sin(timestamp_t_s * (2*np.pi/day))
test['Day cos'] = np.cos(timestamp_t_s * (2*np.pi/day))

# 1 week
test['Week sin'] = np.sin(timestamp_t_s * (2*np.pi/week))
test['Week cos'] = np.cos(timestamp_t_s * (2*np.pi/week))

# Model

Using CatBoost Regression

In [ ]:
X = data[['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5','deg_C','relative_humidity','absolute_humidity','Day sin', 'Day cos', 'Week cos','yhat_monoxide']]
y = data['target_carbon_monoxide']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params={'iterations':25000,
       'learning_rate':0.02,
       'loss_function':'RMSE',
       'eval_metric':'RMSE',
       'verbose':200,
        'od_type':"Iter", #overfit detector
        'od_wait':200, # most recent best iteration to wait before 500
       'use_best_model':True}

modelc = CatBoostRegressor(**params)
modelc.fit(X_train, y_train, eval_set=(X_test, y_test),
         use_best_model=True,
         plot=True,
         early_stopping_rounds=100,
         verbose=False)

In [ ]:
shape_values = modelc.get_feature_importance(Pool(X_test, label=y_test),type="ShapValues")
expected_value = shape_values[0,-1]
shape_values = shape_values[:,:-1]
shap.summary_plot(shape_values, X_test)

In [ ]:
X = data[['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5','relative_humidity','absolute_humidity','Day sin', 'Day cos','Week sin','Week cos','yhat_benzene_oxide']]
y = data['target_benzene']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

params={'iterations':20000,
       'learning_rate':0.01,
       'loss_function':'RMSE',
       'eval_metric':'RMSE',
       'verbose':200,
        'od_type':"Iter", #overfit detector
        'od_wait':200, # most recent best iteration to wait before 500
       'use_best_model':True}

modelb = CatBoostRegressor(**params)
modelb.fit(X_train, y_train, eval_set=(X_test, y_test),
         use_best_model=True,
         plot=True,
         early_stopping_rounds=100,
         verbose=False)

In [ ]:
shape_values = modelb.get_feature_importance(Pool(X_test, label=y_test),type="ShapValues")
expected_value = shape_values[0,-1]
shape_values = shape_values[:,:-1]
shap.summary_plot(shape_values, X_test)

In [ ]:
X = data[['sensor_1','sensor_2','sensor_3','sensor_4','sensor_5','deg_C','relative_humidity','absolute_humidity','Day sin', 'Day cos', 'Week cos','yhat_nitrogen_oxide']]
#X = data[['sensor_2']]
y = data['target_nitrogen_oxides']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params={'iterations':20000,
       'learning_rate':0.02,
       'loss_function':'RMSE',
       'eval_metric':'RMSE',
       'verbose':200,
        'od_type':"Iter", #overfit detector
        'od_wait':200, # most recent best iteration to wait before 500
       'use_best_model':True}

#model = Pipeline([('LR',CatBoostRegressor(**params))])
modeln = CatBoostRegressor(**params)
modeln.fit(X_train, y_train, eval_set=(X_test, y_test),
         use_best_model=True,
         plot=True,
         early_stopping_rounds=200,
         verbose=False)

In [ ]:
shape_values = modeln.get_feature_importance(Pool(X_test, label=y_test),type="ShapValues")
expected_value = shape_values[0,-1]
shape_values = shape_values[:,:-1]
shap.summary_plot(shape_values, X_test)

In [ ]:
my_submission = pd.DataFrame({'date_time': test.index, 'target_carbon_monoxide': modelc.predict(test), 'target_benzene':modelb.predict(test), 'target_nitrogen_oxides':modeln.predict(test)})

In [ ]:
data = data.reset_index().rename(columns = {'ds':'date_time'})
my_submission = pd.concat([data[['date_time','target_carbon_monoxide','target_benzene','target_nitrogen_oxides']][-1:],my_submission]).reset_index(drop=True)

In [ ]:
my_submission.to_csv('submission.csv', index=False)